In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\arka\\OneDrive\\Documents\\PYTHON_R'

In [3]:
os.chdir('C:/Users/arka/OneDrive/Arka_PhD/Blockchain')

In [7]:
import csv
import hashlib
import pandas as pd
from cryptography.fernet import Fernet

# Function to create a hash of a row
def create_row_hash(row):
    row_str = ''.join(str(cell) for cell in row)
    return hashlib.sha256(row_str.encode()).hexdigest()

# Load the patient data from the CSV file
def load_patient_data():
    try:
        data = pd.read_csv("patient_data.csv")
    except FileNotFoundError:
        data = pd.DataFrame(columns=["patient_id", "age", "weight", "height", "hash"])
    return data

# Load the patient IDs and their corresponding hashes from the CSV file
def load_patient_hashes():
    try:
        patient_hashes = pd.read_csv("patient_hashes.csv")
    except FileNotFoundError:
        patient_hashes = pd.DataFrame(columns=["patient_id", "hash"])
    return patient_hashes

# Function to add a new patient to the CSV file
def add_patient():
    patient_id = input("Enter patient ID: ")
    age = input("Enter age: ")
    weight = input("Enter weight: ")
    height = input("Enter height: ")

    # Create a new row for the patient
    new_patient = [patient_id, age, weight, height, '']
    
    # Load the existing patient data
    data = load_patient_data()

    # Calculate the hash of the new row
    new_hash = create_row_hash(new_patient)
    new_patient[-1] = new_hash

    # Append the new patient to the data
    data = pd.concat([data, pd.DataFrame([new_patient], columns=data.columns)], ignore_index=True)
    
    # Save the updated data to the CSV file
    data.to_csv("patient_data.csv", index=False)
    print("Patient added successfully.")

    # Load the patient hashes
    patient_hashes = load_patient_hashes()

    # Check if the patient ID is in the patient_hashes
    if patient_id in patient_hashes['patient_id'].values:
        # Update the hash in patient_hashes
        patient_hashes.loc[patient_hashes['patient_id'] == patient_id, 'hash'] = new_hash
    else:
        # Add a new row for the patient in patient_hashes
        #patient_hashes = patient_hashes.append({'patient_id': patient_id, 'hash': new_hash}, ignore_index=True)
        new_patient_hash = pd.DataFrame({'patient_id': [patient_id], 'hash': [new_hash]})
        patient_hashes = pd.concat([patient_hashes, new_patient_hash], ignore_index=True)

    # Save the updated patient hashes
    patient_hashes.to_csv("patient_hashes.csv", index=False)

# Function to check data integrity
def check_data_integrity():
    data = load_patient_data()
    patient_hashes = load_patient_hashes()
    compromised_rows = []
    for index, row in data.iterrows():
        expected_hash = create_row_hash(row[:-1])
        if row['hash'] != expected_hash:
            compromised_rows.append(row['patient_id'])

    if len(compromised_rows) == 0:
        print("Data integrity is intact. No unauthorized changes detected.")
    else:
        print("Data integrity has been compromised in the following rows:")
        print(compromised_rows)

# Function to encrypt and save columns in a separate CSV file
def encrypt_and_save_columns():
    # Generate a Fernet key for encryption
    key = Fernet.generate_key()
    

    # Save and securely store this key for decryption
    with open("fernet_key.key", "wb") as key_file:
        key_file.write(key)

    # Load the patient data
    data = load_patient_data()

    # Define the columns to be encrypted
    columns_to_encrypt = ["age", "weight", "height"]

    # Create a Fernet cipher with the key
    cipher = Fernet(key)

    # Encrypt specified columns
    for col in columns_to_encrypt:
        data[col] = data[col].apply(lambda x: cipher.encrypt(str(x).encode()).decode())

    # Save the encrypted data to a separate CSV file
    data[["patient_id"] + columns_to_encrypt].to_csv("encrypted_patient_data.csv", index=False)
    print("Columns encrypted and saved successfully.")

# Main program
while True:
    print("1. Add new patient")
    print("2. Check Data Integrity")
    print("3. Encrypt and Save Columns")
    print("4. Exit")
    choice = input("Enter your choice: ")

    if choice == '1':
        add_patient()
    elif choice == '2':
        check_data_integrity()
    elif choice == '3':
        encrypt_and_save_columns()
    elif choice == '4':
        break
    else:
        print("Invalid choice. Please try again.")

1. Add new patient
2. Check Data Integrity
3. Encrypt and Save Columns
4. Exit
Enter your choice: 4


In [6]:
import pandas as pd
from cryptography.fernet import Fernet

# Load the Fernet key from a file
with open("fernet_key.key", "rb") as key_file:
    decryption_key = key_file.read()

# Create a Fernet cipher with the key
cipher = Fernet(decryption_key)

# Load the encrypted CSV file
encrypted_data = pd.read_csv("encrypted_patient_data.csv")

# Define the columns to decrypt
columns_to_decrypt = ["age", "weight", "height"]

# Decrypt the specified columns
for col in columns_to_decrypt:
    encrypted_data[col] = encrypted_data[col].apply(lambda x: cipher.decrypt(x.encode()).decode())

# Save the decrypted data to a new CSV file
decrypted_file = "decrypted_patient_data.csv"
encrypted_data.to_csv(decrypted_file, index=False)
print(f"Decrypted data has been saved to '{decrypted_file}'.")

Decrypted data has been saved to 'decrypted_patient_data.csv'.
